# Homework 1

## FINM 37500: Fixed Income Derivatives

### Mark Hendricks

#### Winter 2024

# Context

For use in these problems, consider the data below, discussed in Veronesi's *Fixed Income Securities* Chapters 9, 10.
* interest-rate tree
* current term structure

In [498]:
import numpy as np
import pandas as pd

In [499]:
rate_tree = pd.DataFrame({'0':[.0174,np.nan],'0.5':[.0339,.0095]})
rate_tree.columns.name = 'time $t$'
rate_tree.index.name = 'node'
rate_tree.style.format('{:.2%}',na_rep='')

time $t$,0,0.5
node,,
0,1.74%,3.39%
1,,0.95%


The "tree" is displayed as a pandas dataframe, so it does not list "up" and "down" for the rows but rather an index of nodes. The meaning should be clear.

In [500]:
term_struct = pd.DataFrame({'maturity':[.5,1,1.5],'price':[99.1338,97.8925,96.1462]})
term_struct['continuous ytm'] = -np.log(term_struct['price']/100) / term_struct['maturity']
term_struct.set_index('maturity',inplace=True)
term_struct.style.format({'price':'{:.4f}','continuous ytm':'{:.2%}'}).format_index('{:.1f}')

,price,continuous ytm
maturity,,
0.5,99.1338,1.74%
1.0,97.8925,2.13%
1.5,96.1462,2.62%


This is the current term-structure observed at $t=0$.

# 1. Pricing a Swap



### 1.1 
Calculate the tree of bond prices for the 2-period, $T=1$, bond.


In [501]:
class tree_convention:
    UP=1
    DOWN=-1
    ORIGIN=0
    TERMINAL=2

class interest_node():
    def __init__(self, period, rate, state):
        self.period = period
        self.rate = rate
        self.state = state

    def __repr__(self):
        return 'rate: {:.2%}\nperiod: {}\nstate: {}'.format(self.rate,self.period,self.state)
    
class Interest_Tree():
    def __init__(self, periods, rates):
        self.periods = periods
        self.rates = rates
        self.levels = []
        self.create_tree()
    
    def create_tree(self):
        self.levels.append([interest_node(0,self.rates[0][0],0)])
        for i in range(1,len(self.periods)):
            tlevel=[]
            for j in range(len(self.levels[i-1])):
                node=interest_node(self.periods[i],self.rates[i][(j*2)],tree_convention.UP)
                tlevel.append(node)
                node=interest_node(self.periods[i],self.rates[i][(2*j)+1],tree_convention.DOWN)
                tlevel.append(node)
            self.levels.append(tlevel)
    
    def __repr__(self):
        return 'Interest_tree: {}'.format(self.levels)
                

class bond_node():
    def __init__(self, period, price, state, maturity):
        self.period = period
        self.price = price
        self.state = state
        self.maturity = maturity
        self.rn_probUP=1/2
        self.rn_probDOWN=1/2

    def __repr__(self):
        return 'period: {}\nstate: {}\nprice: {:.2f}\nmaturity: {}'.format(self.period, self.state, self.price, self.maturity)

class Binary_Bond_Tree():
    def __init__(self, periods, price, face_value, interest_tree, maturity):
        self.periods = np.array(periods)
        self.price = price
        self.face_value = face_value
        self.levels = []
        self.interest_tree = interest_tree
        self.maturity = maturity
        self.create_tree()
        self.risk_neutral_tree()

    def create_tree(self):
        tprices=[[bond_node(self.maturity,self.face_value,tree_convention.TERMINAL,self.maturity)]*(2**len(self.periods))]
        self.levels.append(tprices[0])
        for i in range(1,len(self.periods)):
            tprices=[]
            for j in range(int(len(self.levels[i-1])/4)):
                tprice=self.levels[i-1][4*j].price
                tpriceUP=float(tprice)*np.exp(-float(self.interest_tree.levels[-i][(2*j)].rate)*(float(self.levels[i-1][0].period)-float(self.periods[-i])))
                tprices.append(bond_node(self.periods[-i],tpriceUP,tree_convention.UP,self.maturity))
                tprice=self.levels[i-1][(4*j)+3].price
                tpriceDOWN=float(tprice)*np.exp(-float(self.interest_tree.levels[-i][(2*j)+1].rate)*(float(self.levels[i-1][0].period)-float(self.periods[-i])))
                tprices.append(bond_node(self.periods[-i],tpriceDOWN,tree_convention.DOWN,self.maturity))
            self.levels.append(tprices)
        self.levels.append([bond_node(0,self.price,tree_convention.ORIGIN,self.maturity)])
        self.levels=self.levels[::-1]

    def risk_neutral_tree(self):
        for i in range(len(self.levels)-2):
            for j in range(max(int(len(self.levels[i])/2),1)):
                adprice=self.levels[i][j].price*np.exp(float(self.interest_tree.levels[i][j].rate)*(float(self.periods[i+1])-float(self.periods[i])))
                pup=(adprice-self.levels[i+1][(j*2)+1].price)/(self.levels[i+1][j*2].price-self.levels[i+1][(j*2)+1].price)
                self.levels[i][j].rn_probUP=pup
                self.levels[i][j].rn_probDOWN=1-pup

        
    def __repr__(self):
        tree_str = 'Binary_Bond_tree:\n'
        tree_str += '                               Maturity: {}\n'.format(self.maturity)
        for level_index, level in enumerate(self.levels[:-1]):
            level_str = '  ' * level_index
            for node in level:
                node_str = '[state: {}, price: {:.2f}]'.format(
                    node.state, node.price)
                level_str += node_str + '  '
            tree_str += 'period: {} '.format(node.period) + level_str.strip() + '\n'
        return tree_str.strip()
            

In [502]:
class tree_convention:
    UP=1
    DOWN=-1
    ORIGIN=0
    TERMINAL=2

class interest_node():
    def __init__(self, period, rate, state):
        self.period = period
        self.rate = rate
        self.state = state

    def __repr__(self):
        return 'rate: {:.2%}\nperiod: {}\nstate: {}'.format(self.rate,self.period,self.state)
    
class Interest_Tree():
    def __init__(self, periods, rates):
        self.periods = periods
        self.rates = rates
        self.levels = []
        self.create_tree()
    
    def create_tree(self):
        self.levels.append([interest_node(0,self.rates[0][0],0)])
        for i in range(1,len(self.periods)):
            tlevel=[]
            for j in range(len(self.levels[i-1])):
                node=interest_node(self.periods[i],self.rates[i][(j*2)],tree_convention.UP)
                tlevel.append(node)
                node=interest_node(self.periods[i],self.rates[i][(2*j)+1],tree_convention.DOWN)
                tlevel.append(node)
            self.levels.append(tlevel)
    
    def __repr__(self):
        return 'Interest_tree: {}'.format(self.levels)
                

class bond_node():
    def __init__(self, period, price, state, maturity,expected_price=None):
        self.period = period
        self.price = price
        self.state = state
        self.maturity = maturity
        self.expected_price=expected_price
        self.rn_probUP=1/2
        self.rn_probDOWN=1/2

    def __repr__(self):
        return 'period: {}\nstate: {}\nprice: {:.2f}\nmaturity: {}'.format(self.period, self.state, self.price, self.maturity)

class Binary_Bond_Tree():
    def __init__(self, periods, price, face_value, interest_tree, maturity):
        self.periods = np.array(periods)
        self.price = price
        self.face_value = face_value
        self.levels = []
        self.interest_tree = interest_tree
        self.maturity = maturity
        self.populate_expected_prices()
        self.create_terminal_conditions()
        self.populate_risk_neutral_probabilities_prices()

    def populate_expected_prices(self):
        self.levels.append([bond_node(0,self.price,tree_convention.ORIGIN,self.maturity,\
            self.price*np.exp(float(self.interest_tree.levels[0][0].rate)*float(self.periods[1])))])
        for i in range(1,len(self.periods)-1):
            tlevel=[]
            for j in range(len(self.levels[i-1])):
                tprice=float(self.levels[i-1][j].expected_price)
                tpriceUP=tprice*np.exp(float(self.interest_tree.levels[i][j].rate)*(float(self.periods[i+1])-float(self.periods[i])))
                tpriceDOWN=tprice*np.exp(float(self.interest_tree.levels[i][j+1].rate)*(float(self.periods[i+1])-float(self.periods[i])))
                tlevel.append(bond_node(self.periods[i],0,tree_convention.UP,self.maturity,tpriceUP))
                tlevel.append(bond_node(self.periods[i],0,tree_convention.DOWN,self.maturity,tpriceDOWN))
            self.levels.append(tlevel)

    def create_terminal_conditions(self):
        tprices_end=[[bond_node(self.maturity,self.face_value,tree_convention.TERMINAL,self.maturity)]*(2**len(self.periods))]
        tprices=[]
        for j in range(int(len(tprices_end[0])/4)):
            tpriceUP=self.face_value*np.exp(-float(self.interest_tree.levels[-1][(2*j)].rate)*(float(self.maturity)-float(self.periods[-1])))
            tprices.append(bond_node(self.periods[-1],tpriceUP,tree_convention.UP,self.maturity))
            tpriceDOWN=self.face_value*np.exp(-float(self.interest_tree.levels[-1][(2*j)+1].rate)*(float(self.maturity)-float(self.periods[-1])))
            tprices.append(bond_node(self.periods[-1],tpriceDOWN,tree_convention.DOWN,self.maturity))
        self.levels.append(tprices)
        self.levels.append(tprices_end[0])

    def populate_risk_neutral_probabilities_prices(self):
        for i in range(-3,-len(self.levels)-1,-1):
            for ii in range(len(self.levels[i])):
                probUP=(self.levels[i][ii].expected_price-self.levels[i+1][ii*2+1].price)/(self.levels[i+1][ii*2].price-self.levels[i+1][ii*2+1].price)
                self.levels[i][ii].rn_probUP=probUP
                self.levels[i][ii].rn_probDOWN=1-probUP
                self.levels[i][ii].price=self.levels[i][ii].expected_price*\
                    np.exp(-float(self.interest_tree.levels[i+1][ii].rate)*(float(self.periods[i+2])-float(self.periods[i+1])))

    def __repr__(self):
        tree_str = 'Binary_Bond_tree:\n'
        tree_str += '                               Maturity: {}\n'.format(self.maturity)
        for level_index, level in enumerate(self.levels[:-1]):
            level_str = '  ' * level_index
            for node in level:
                node_str = '[state: {}, price: {:.2f}]'.format(
                    node.state, node.price)
                level_str += node_str + '  '
            tree_str += 'period: {} '.format(node.period) + level_str.strip() + '\n'
        return tree_str.strip()
            

In [503]:
interest_tree=Interest_Tree(rate_tree.columns,rate_tree.values.T)
bond_tree=Binary_Bond_Tree(rate_tree.columns,term_struct.loc[1,'price'],100,interest_tree,1)
print(bond_tree)

Binary_Bond_tree:
                               Maturity: 1
period: 0 [state: 0, price: 97.89]
period: 0.5 [state: 1, price: 98.32]  [state: -1, price: 99.53]



### 1.2 
What is the risk-neutral probability of an upward movement of interest rates at $t=.5$?


In [504]:
print('Risk Neutral Probability Up: ', round(bond_tree.levels[0][0].rn_probUP*100,2),'%')

Risk Neutral Probability Up:  64.49 %



## The option contract

Consider a single-period swap that pays at time period 1 ($t=0.5$), the expiration payoff (and thus terminal value) is
* Payoff = $\frac{100}{2}(r_1 −c)$
* with $c=2\%$
* payments are semiannual

Take the viewpoint of a fixed-rate payer, floating rate receiver.



### 1.3 
What is the replicating trade using the two bonds (period 1 and period 2)?


In [505]:
flows_swap=(100/2)*(np.array([x.rate for x in interest_tree.levels[1]]) - 0.02)
beta=(flows_swap[0]-flows_swap[1])/(bond_tree.levels[1][0].price-bond_tree.levels[1][1].price)
alpha=(flows_swap[0]-beta*bond_tree.levels[1][0].price)/100
print('Alpha: ', alpha)
print('Beta: ', beta)

Alpha:  1.0008624033025624
Beta:  -1.0109028054163192


In [506]:
print('Positions: \nBonds period 1: ', round(alpha,4), '\nBonds period 2: ', round(beta,4))

Positions: 
Bonds period 1:  1.0009 
Bonds period 2:  -1.0109



### 1.4 
What is the price of the swap?

In [507]:
swap_p0=(alpha*term_struct.loc[0.5,'price'])+(beta*term_struct.loc[1,'price'])
print('Price of the swap [USD]: ', round(swap_p0,4))

Price of the swap [USD]:  0.2595


# 2. Using the Swap as the Underlying
As in the note, W.1, consider pricing the followign interest-rate option,
* Payoff is $100\max(r_K-r_1,0)$
* strike is $r_K$ is 2\%
* expires at period 1, ($t=0.5$) 

Unlike the note, price it with the swap used as the underlying, not the two-period ($t=1$) bond. You will once again use the period-1 ($t=0.5$) bond as the cash account for the no-arbitrage pricing.

So instead of replicating the option with the two treasuries, now you're replicating/pricing it with a one-period bond and two-period swap.



### 2.1
Display the tree of swap prices.


In [508]:
interest_tree

Interest_tree: [[rate: 1.74%
period: 0
state: 0], [rate: 3.39%
period: 0.5
state: 1, rate: 0.95%
period: 0.5
state: -1]]

In [509]:
flows=[100*max(y,0) for y in 0.02-np.array([x.rate for x in interest_tree.levels[1]])]
flows

[0, 1.05]

In [510]:
print('Swap prices:')
print('Period 0 = ', round(swap_p0,4))
print('Period 0.5 = ', 'up: ', round(flows_swap[0],4), '\ down: ', round(flows_swap[1],4))

Swap prices:
Period 0 =  0.2595
Period 0.5 =  up:  0.695 \ down:  -0.525



### 2.2
What is the risk-neutral probability of an upward movement at $t=.5$ implied by the underlying swap tree? 

Is this the same as the risk-neutral probability we found when the bond was used as the underlying?


In [511]:
probability_swap_up=((swap_p0*interest_tree.levels[0][0].rate)-flows_swap[1])/(flows_swap[0]-flows_swap[1])
print('Probability of swap up: ', round(probability_swap_up,4))
print('Probability found through the bond: ', round(bond_tree.levels[0][0].rn_probUP,4))

Probability of swap up:  0.434
Probability found through the bond:  0.6449



### 2.3
What is the price of the rate option? Is it the same as we calculated in the note, W.1.?

In [512]:
beta_swap=(flows[0]-flows[1])/(flows_swap[0]-flows_swap[1])
alpha_swap=(flows[0]-beta_swap*flows_swap[0])/100
print('Alpha swap: ', round(alpha_swap,4))
print('Beta swap: ', round(beta_swap,4))
print('\nPositions swap: \nBonds period 1: ', round(alpha_swap,4), '\nSwaps: ', round(beta_swap,4))
print('\nPrice of the instrument [USD]: ', round(alpha_swap*term_struct.loc[0.5,'price']+beta_swap*swap_p0, 4))

Alpha swap:  0.006
Beta swap:  -0.8607

Positions swap: 
Bonds period 1:  0.006 
Swaps:  -0.8607

Price of the instrument [USD]:  0.3696


# 3. Pricing a Call on a Bond

Try using the same tree to price a call on the period-2 bond, (1-year), at period 1 (6-months).
* Payoff = $\max(P_{1|2}-K,0)$
* Strike = \$99.00

### 3.1 
What is the replicating trade using the two bonds (period 1 and period 2) as above? (That is, we are no longer using the swap as the underlying.)


In [513]:
K=99
flows_call=[max(y, 0) for y in np.array([x.price for x in bond_tree.levels[1]])-K]
flows_call

[0, 0.5261263409211807]

In [514]:
beta_call=(flows_call[0]-flows_call[1])/(bond_tree.levels[1][0].price-bond_tree.levels[1][1].price)
alpha_call=(flows_call[0]-beta_call*bond_tree.levels[1][0].price)/100
print('Alpha call: ', round(alpha_call,4))
print('Beta call: ', round(beta_call,4))
print('\nPositions call: \nBonds period 1: ', round(alpha_call,4), '\nBonds period 2: ', round(beta_call,4))

Alpha call:  -0.4286
Beta call:  0.436

Positions call: 
Bonds period 1:  -0.4286 
Bonds period 2:  0.436



### 3.2 
What is the price of the European call option? 
* expiring at $T=.5$ 
* written on the bond maturing in 2 periods, ($t=1$)

In [515]:
print('Price of the instrument [USD]: ', round(alpha_call*term_struct.loc[0.5,'price']+beta_call*term_struct.loc[1,'price'], 4))

Price of the instrument [USD]:  0.1852


# 4 Two-Period Tree

Consider an expanded, **2 period** tree. (Two periods of uncertainty, so with the starting point, three periods total.)

In [516]:
new_col = pd.Series([.05,.0256,.0011],name='1')
rate_tree_multi = pd.concat([rate_tree,new_col],ignore_index=True,axis=1)
rate_tree_multi.columns = pd.Series(['0','0.5','1'],name='time $t$')
rate_tree_multi.index.name = 'node'
rate_tree_multi.style.format('{:.2%}',na_rep='')

time $t$,0,0.5,1
node,,,
0,1.74%,3.39%,5.00%
1,,0.95%,2.56%
2,,,0.11%


### 4.1

Calculate and show the tree of prices for the 3-period bond, $T=1.5$.


In [517]:
rate_tree_multi

time $t$,0,0.5,1
node,,,
0,0.0174,0.0339,0.0500
1,NaN,0.0095,0.0256
2,NaN,NaN,0.0011


In [518]:
ad_rate_tree_multi = rate_tree_multi.copy()
col1 = list(ad_rate_tree_multi['1'])
tcol=[]
for i in range(len(col1)):
    tcol.append(col1[i])
    if i==1:
        tcol.append(col1[i])

ad_rate_tree_multi.loc[len(ad_rate_tree_multi)] = np.nan
ad_rate_tree_multi.loc[:,'1'] = tcol
ad_rate_tree_multi

time $t$,0,0.5,1
node,,,
0,0.0174,0.0339,0.0500
1,NaN,0.0095,0.0256
2,NaN,NaN,0.0256
3,NaN,NaN,0.0011


In [519]:
interest_tree_multi=Interest_Tree(ad_rate_tree_multi.columns,ad_rate_tree_multi.values.T)
bond_tree_multi=Binary_Bond_Tree(ad_rate_tree_multi.columns,term_struct.loc[1.5,'price'],100,interest_tree_multi,1.5)
print(bond_tree_multi)

Binary_Bond_tree:
                               Maturity: 1.5
period: 0 [state: 0, price: 96.15]
period: 0.5 [state: 1, price: 96.99]  [state: -1, price: 96.99]
period: 1 [state: 1, price: 97.53]  [state: -1, price: 98.73]  [state: 1, price: 98.73]  [state: -1, price: 99.95]



### 4.2
Report the risk-neutral probability of an up movement at $t=1$.

(The risk-neutral probability of an up movement at $t=0.5$ continues to be as you calculated in 2.3.


In [520]:
print('Risk Neutral Probability Up: ', round(bond_tree_multi.levels[1][0].rn_probUP*100,2),'%')

Risk Neutral Probability Up:  7.01 %



### 4.3
Calculate the price of the European **call** option?
* expiring at $T=1$ 
* written on the bond maturing in 3 periods, ($t=1.5$)



### 4.4
Consider a finer time grid. Let $dt$ in the tree now be 1/30 instead of 0.5.

Using this smaller time step, compute the $t=0$ price of the following option:
* option expires at $t=1$
* written on bond maturing at $t=1.5

# 5 American Style
### 5.1
Use the two-period tree from part 4, but this time to price an American-style **put** option.

Use a grid of $dt=.5$.
* What is its value at $t=0$?
* Which nodes would you exercise it early?

### 5.2
Change the grid to $dt=1/30$, as in 4.4. 
* What is its value at $t=0$?
* Make a visualization showing which nodes have early exercise. (I suggest using a dataframe and the `heatmap` from `seaborn`.